# Calculating Indices using Xclim

This notebook was designed to compute Extreme Precipitation Climate Indices computations (EPCI) using [XClim]('https://xclim.readthedocs.io/en/stable/notebooks/example.html#Creating-xarray-datasets') based in Python Xarray.

---

 - Author:          
                    Luis F Patino Velasquez - MA
 - Date:            
                    Jun 2020
 - Version:         
                    1.0
 - Notes:            
                    Files used in this notebook are in netCDF format
 - Jupyter version: 
                    jupyter core     : 4.7.1
                    jupyter-notebook : 6.4.0
                    qtconsole        : 5.1.1
                    ipython          : 7.25.0
                    ipykernel        : 6.0.3
                    jupyter client   : 6.1.12
                    jupyter lab      : 3.0.16
                    nbconvert        : 6.1.0
                    ipywidgets       : 7.6.3
                    nbformat         : 5.1.3
                    traitlets        : 5.0.5
 - Python version:  
                    3.8.5 

---

## Main considerations

* all the files used in this process underwent a modification in the precipitation values due to PRCPtot funcion of Xclim. To ensure only precipitation values >= 1mm were used in the indices computation the following NCO command was used over all the files for HadUK-grid, IMERG and ERA5 `ncap2 -s 'where(precipitationCal <= 1.0) precipitationCal=0.0;' -O in.nc out.nc`
* the precipitation units for the sample dataset need to be changed to mm d-1, this was done using the following command in nco `ncatted -O -a units,precip,m,c,"mm d-1" in.nc`
* The list of available frequency is given in the link below, but the most often used are:
    - YS: annual starting in January
    - YS-JUL: annual starting in July
    - MS: monthly
    - QS-DEC: seasonal starting in December

## Setting Python Modules

In [ ]:
# Imports for xclim and xarray
import xclim as xc
import pandas as pd
import numpy as np
from numpy import exp
import xarray as xr
import scipy.stats as stats
from scipy.stats import rankdata
from scipy.stats import norm
from scipy.stats import anderson
from scipy.stats import shapiro
from scipy.stats import boxcox
from math import ceil

# File handling libraries
import time
import tempfile
from pathlib import Path

# import plotting stuff
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import statsmodels.api as sm
%matplotlib inline
# Set some plotting defaults
plt.rcParams['figure.figsize'] = (15, 11)
plt.rcParams['figure.dpi'] = 100

# Global variables
fldr_out = Path('/mnt/d/MRes_dataset/active_data/102_precp/')
sep = ('''------------\n------------''')

# Define years list between 1991 and 2010
str_yr = 2001
end_yr = 2019
yrs_lt = list(range(str_yr, end_yr + 1, 1))
print(sep)

In [ ]:
def savingFile(file_name, data_xarray):
    """
    Save image output in folder
    :file_name: integer
    :data_xarray: xarray
    """
    # Check if file already exist
    check = Path(fldr_out / file_name)
    if check.is_file() is False:
        # Saving file with annual precipitations
        xclim_indices = Path(fldr_out / file_name)
        print ('saving to ', xclim_indices)
        data_xarray.to_netcdf(path=xclim_indices)
        print ('finished saving')
    else:
        print ('{} already exist - try using a different file name'.format(file_name))

# ERA5

## Loading and reading nc file

In [ ]:
# Set directory to read and for outputs
fldr_src = Path('/mnt/d/MRes_dataset/search_data/era_copernicus_uk_xclim/')

# Create list with files
fls_lst = fldr_src.glob('**/era5_copernicus_DAY_prcp_*')

# Load multiple NetCDFs into a single xarray.Dataset
dataset_ERA = xr.open_mfdataset(paths=fls_lst, combine='by_coords', parallel=True)
dataset_ERA

## Example of precipitation data for January for all years

In [ ]:
# Use year list to plot - It needs to be between 1991 and 2010

fig, axs = plt.subplots(4, 5, sharex=True, sharey=True)
for i, ax in enumerate(axs.flat):
    if i < len(yrs_lt):
        year_str = str(yrs_lt[i]) + '-01-01' # This can be modified to add other month or date
        dataset_ERA['tp'].sel(time=year_str).plot(ax= ax, x='longitude', y='latitude', cmap='YlGnBu')
        ax.set_title('Precipitation ' + str(yrs_lt[i]) + '-01-01')
        # Remove axis label to enable common label at the end
        ax.set_ylabel('')
        ax.set_xlabel('')
    else:
        break

# set labels
plt.setp(axs[-1, :], xlabel='Longitude')
plt.setp(axs[:, 0], ylabel='Latitude')

# Make sure it show a nice layout avoiding overlapping
plt.tight_layout()

## Xclim computation

- The list of indices can be found [here](https://xclim.readthedocs.io/en/latest/indicators_api.html)
- The indicator chosen are:
    R10mm / R20mm / CDD / CWD / SDII / RX1day / RX5day / R95p / R99p / PRCPTOT

In [ ]:
# Indicators
# this silence the user warning - remove if you wan to see them
xc.set_options(cf_compliance='log')

print('Doing r10mm')
# R10mm
r_10 = xc.indicators.icclim.R10mm(dataset_ERA.tp, freq='QS-DEC')
print('Doing r20mm')
# R20mm
r_20 = xc.indicators.icclim.R20mm(dataset_ERA.tp, freq='QS-DEC')
print('Doing CDD')
# CDD
cdd = xc.indicators.icclim.CDD(dataset_ERA.tp, freq='QS-DEC')
print('Doing CWD')   
# CWD
cwd = xc.indicators.icclim.CWD(dataset_ERA.tp, freq='QS-DEC')
print('Doing SDII')
# SDII
sdii = xc.indicators.icclim.SDII(dataset_ERA.tp, freq='QS-DEC')
print('Doing RX1') 
# RX1day
rx1 = xc.indicators.icclim.RX1day(dataset_ERA.tp, freq='QS-DEC')
print('Doing RX5')
# RX5day
rx5 = xc.indicators.icclim.RX5day(dataset_ERA.tp, freq='QS-DEC')
print('Doing PRCPTOT')
# PRCPTOT
prcp_tot = xc.indicators.atmos.precip_accumulation(dataset_ERA.tp, freq='QS-DEC')
print('Doing R99pTOT')
# R99pTOT
p99 = xc.core.calendar.percentile_doy(dataset_ERA.tp, per=99)
r99 = xc.indicators.icclim.R99pTOT(dataset_ERA.tp, per=p99, freq='QS-DEC')
print('Doing R95pTOT')
# R95pTOT
p95 = xc.core.calendar.percentile_doy(dataset_ERA.tp, per=95)
r95 = xc.indicators.icclim.R95pTOT(dataset_ERA.tp, per=p95, freq='QS-DEC')

# We have created an xarray data-array - 
# We can insert this into an output xr.Dataset object with a copy of the original dataset global attrs
EPI_ERA = xr.Dataset(attrs=dataset_ERA.attrs)

# r99ptot is stored as a different file due to conflicts with the percentile dimension and variables
EPI_ERA_99 = xr.Dataset(attrs=dataset_ERA.attrs)

# p95TOT and p99TOT need to be stored in different data due to percentage field
# Add our climate index as a data variable to the dataset for r95pTOT
EPI_ERA['r10mm'] = r_10
EPI_ERA['r20mm'] = r_20
EPI_ERA['cdd'] = cdd
EPI_ERA['cwd'] = cwd
EPI_ERA['sdii'] = sdii
EPI_ERA['rx1day'] = rx1
EPI_ERA['rx5day'] = rx5
EPI_ERA['prcptot'] = prcp_tot
EPI_ERA['r95ptot'] = r95

# Add our climate index as a data variable to the dataset for r99pTOT
EPI_ERA_99['r10mm'] = r_10
EPI_ERA_99['r20mm'] = r_20
EPI_ERA_99['cdd'] = cdd
EPI_ERA_99['cwd'] = cwd
EPI_ERA_99['sdii'] = sdii
EPI_ERA_99['rx1day'] = rx1
EPI_ERA_99['rx5day'] = rx5
EPI_ERA_99['prcptot'] = prcp_tot
EPI_ERA_99['r99ptot'] = r99

# GPM-IMERG

## Loading and reading nc file

In [ ]:
# Set directory to read and for outputs
fldr_src = Path('/mnt/d/MRes_dataset/search_data/gpm_imerg_nasa_uk_xclim/')

# Create list with files
fls_lst = fldr_src.glob('**/*')


# Load multiple NetCDFs into a single xarray.Dataset
dataset_IMERG = xr.open_mfdataset(paths=fls_lst, combine='by_coords', parallel=True)
dataset_IMERG

## Xclim computation

- The list of indices can be found [here](https://xclim.readthedocs.io/en/latest/indicators_api.html)
- The indicator chosen are:
    R10mm / R20mm / CDD / CWD / SDII / RX1day / RX5day / R95p / R99p / PRCPTOT

In [ ]:
# Indicators
# this silence the user warning - remove if you wan to see them
xc.set_options(cf_compliance='log')
print('Doing r10mm')
# R10mm
r_10 = xc.indicators.icclim.R10mm(dataset_IMERG.precipitationCal, freq='QS-DEC')
print('Doing r20mm')
# R20mm
r_20 = xc.indicators.icclim.R20mm(dataset_IMERG.precipitationCal, freq='QS-DEC')
print('Doing CDD')
# CDD
cdd = xc.indicators.icclim.CDD(dataset_IMERG.precipitationCal, freq='QS-DEC')
print('Doing CWD')    
# CWD
cwd = xc.indicators.icclim.CWD(dataset_IMERG.precipitationCal, freq='QS-DEC')
print('Doing SDII')
# SDII
sdii = xc.indicators.icclim.SDII(dataset_IMERG.precipitationCal, freq='QS-DEC')
print('Doing RX1')
# RX1day
rx1 = xc.indicators.icclim.RX1day(dataset_IMERG.precipitationCal, freq='QS-DEC')
print('Doing RX5')
# RX5day
rx5 = xc.indicators.icclim.RX5day(dataset_IMERG.precipitationCal, freq='QS-DEC')
print('Doing PRCPTOT')
# PRCPTOT
prcp_tot = xc.indicators.atmos.precip_accumulation(dataset_IMERG.precipitationCal, freq='QS-DEC')
print('Doing R95pTOT')
# R95pTOT
p95 = xc.core.calendar.percentile_doy(dataset_IMERG.precipitationCal, per=95)
r95 = xc.indicators.icclim.R95pTOT(dataset_IMERG.precipitationCal, per=p95, freq='QS-DEC')
print('Doing R99pTOT')
# R99pTOT
p99 = xc.core.calendar.percentile_doy(dataset_IMERG.precipitationCal, per=99)
r99 = xc.indicators.icclim.R99pTOT(dataset_IMERG.precipitationCal, per=p99, freq='QS-DEC')

# We have created an xarray data-array - 
# We can insert this into an output xr.Dataset object with a copy of the original dataset global attrs
EPI_IMERG = xr.Dataset(attrs=dataset_IMERG.attrs)
# r99ptot is stored as a different file due to conflicts with the percentile dimension and variables
EPI_IMERG_99 = xr.Dataset(attrs=dataset_IMERG.attrs)

# p95TOT and p99TOT need to be stored in different data due to percentage field
# Add our climate index as a data variable to the dataset for r95pTOT
EPI_IMERG['r10mm'] = r_10
EPI_IMERG['r20mm'] = r_20
EPI_IMERG['cdd'] = cdd
EPI_IMERG['cwd'] = cwd
EPI_IMERG['sdii'] = sdii
EPI_IMERG['rx1day'] = rx1
EPI_IMERG['rx5day'] = rx5
EPI_IMERG['prcptot'] = prcp_tot
EPI_IMERG['r95ptot'] = r95

# Add our climate index as a data variable to the dataset for r99pTOT
EPI_IMERG_99['r10mm'] = r_10
EPI_IMERG_99['r20mm'] = r_20
EPI_IMERG_99['cdd'] = cdd
EPI_IMERG_99['cwd'] = cwd
EPI_IMERG_99['sdii'] = sdii
EPI_IMERG_99['rx1day'] = rx1
EPI_IMERG_99['rx5day'] = rx5
EPI_IMERG_99['prcptot'] = prcp_tot
EPI_IMERG_99['r99ptot'] = r99

# HadUK-Grid

## Loading and reading nc file

In [ ]:
# Set directory to read and for outputs
fldr_src = Path('/mnt/d/MRes_dataset/active_data/106_precp')

# Create list with files
fls_lst = fldr_src.glob('**/*')


# Load multiple NetCDFs into a single xarray.Dataset
dataset_HADUK = xr.open_dataset(Path(fldr_src / 'rainfall_hadukgrid_uk_5km_day_WGS84LatLon_2001-2019.nc'), decode_timedelta=False)
dataset_HADUK

## Xclim computation

- The list of indices can be found [here](https://xclim.readthedocs.io/en/latest/indicators_api.html)
- The indicator chosen are:
    R10mm / R20mm / CDD / CWD / SDII / RX1day / RX5day / R95p / R99p / PRCPTOT

In [ ]:
# Indicators
# this silence the user warning - remove if you wan to see them
xc.set_options(cf_compliance='log')

print('Doing r10mm')
# R10mm
r_10 = xc.indicators.icclim.R10mm(dataset_HADUK.rainfall, freq='QS-DEC')
print('Doing r20mm')
# R20mm
r_20 = xc.indicators.icclim.R20mm(dataset_HADUK.rainfall, freq='QS-DEC')
print('Doing CDD')
# CDD
cdd = xc.indicators.icclim.CDD(dataset_HADUK.rainfall, freq='QS-DEC')
# cdd = xc.indicators.cf.cdd(dataset_HADUK.rainfall, freq='QS-DEC')  
print('Doing CWD')
# # CWD
cwd = xc.indicators.icclim.CWD(dataset_HADUK.rainfall, freq='QS-DEC')
print('Doing SDII')
# # SDII
sdii = xc.indicators.icclim.SDII(dataset_HADUK.rainfall, freq='QS-DEC')
print('Doing RX1')
# # RX1day
rx1 = xc.indicators.icclim.RX1day(dataset_HADUK.rainfall, freq='QS-DEC')
print('Doing RX5')
# # RX5day
rx5 = xc.indicators.icclim.RX5day(dataset_HADUK.rainfall, freq='QS-DEC')
print('Doing PRCPTOT')
# # PRCPTOT
prcp_tot = xc.indicators.atmos.precip_accumulation(dataset_HADUK.rainfall, freq='QS-DEC')
print('Doing R95')
# # R95p
p95 = xc.core.calendar.percentile_doy(dataset_HADUK.rainfall, per=95)
r95 = xc.indicators.icclim.R95pTOT(dataset_HADUK.rainfall, per=p95, freq='QS-DEC')
print('Doing R99')
# # R95p
p99 = xc.core.calendar.percentile_doy(dataset_HADUK.rainfall, per=99)
r99 = xc.indicators.icclim.R99pTOT(dataset_HADUK.rainfall, per=p99, freq='QS-DEC')

# # We have created an xarray data-array - 
# # We can insert this into an output xr.Dataset object with a copy of the original dataset global attrs
EPI_HADUK = xr.Dataset(attrs=dataset_HADUK.attrs)
# # r99ptot is stored as a different file due to conflicts with the percentile dimension and variables
EPI_HADUK_99 = xr.Dataset(attrs=EPI_HADUK.attrs)

# p95TOT and p99TOT need to be stored in different data due to percentage field
# Add our climate index as a data variable to the dataset for r95pTOT
EPI_HADUK['r10mm'] = r_10
EPI_HADUK['r20mm'] = r_20
EPI_HADUK['cdd'] = cdd
EPI_HADUK['cwd'] = cwd
EPI_HADUK['sdii'] = sdii
EPI_HADUK['rx1day'] = rx1
EPI_HADUK['rx5day'] = rx5
EPI_HADUK['prcptot'] = prcp_tot
EPI_HADUK['r95ptot'] = r95

# Add our climate index as a data variable to the dataset for r99pTOT
EPI_HADUK_99['r10mm'] = r_10
EPI_HADUK_99['r20mm'] = r_20
EPI_HADUK_99['cdd'] = cdd
EPI_HADUK_99['cwd'] = cwd
EPI_HADUK_99['sdii'] = sdii
EPI_HADUK_99['rx1day'] = rx1
EPI_HADUK_99['rx5day'] = rx5
EPI_HADUK_99['prcptot'] = prcp_tot
EPI_HADUK_99['r99ptot'] = r99

# Saving indicators xarrays

> The code below only needs to be run if you want to save the output of xclim as a file

In [ ]:
# # Saving ERA5
savingFile('era5_copernicus_xclimSeason_QSDEC_prcp_2001-2019.nc', EPI_ERA)
savingFile('era5_copernicus_xclimSeasonR99ptot_QSDEC_prcp_2001-2019.nc', EPI_ERA_99)
# print(sep)
savingFile('gpm_imerg_xclimSeason_QSDEC_prcp_2001-2019.nc', EPI_IMERG)
savingFile('gpm_imerg_xclimSeasonR99ptot_QSDEC_prcp_2001-2019.nc', EPI_IMERG_99)
# print(sep)
savingFile('haduk_metoffice_xclimSeason_QSDEC_prcp_2001-2019.nc', EPI_HADUK)
savingFile('haduk_metoffice_xclimSeasonR99ptot_QSDEC_prcp_2001-2019.nc', EPI_HADUK_99)
# print(sep)
